## STEP 1 create a master password and username authentication system

In [1]:
import json
import hashlib

In [12]:
def hash_password(password):
    
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    return hashed_password

def save_master_password(username, password):
    
    try:
        with open("master_passwords.json", "r") as file:
            data = json.load(file)
    except FileNotFoundError:
        data = {}

   
    hashed_password = hash_password(password)
    
    
    data[username] = hashed_password

    
    with open("master_passwords.json", "w") as file:
        json.dump(data, file)

def load_master_password(username):
    try:
        with open("master_passwords.json", "r") as file:
            data = json.load(file)
            hashed_password = data.get(username)
            return hashed_password
    except FileNotFoundError:
        return None
def username_exists(username):
    try:
        with open("master_passwords.json", "r") as file:
            data = json.load(file)
            return username in data
    except FileNotFoundError:
        return False


In [13]:
while True:    
    choice = input("Enter 1 to login to existing account, or 2 to sign up: ")
    if int(choice) == 1:

        username = input("Enter Master Username: ")
        password = input("Enter Master password: ")
        hashed_password = load_master_password(username)
        if hashed_password == hash_password(password):
            print("User found:", username)
            break
        else:
            print("Please Check your Username or Password and try again")
    
    if int(choice) == 2:
        username = input("Enter New Master Username: ")
        password = input("Enter New Master password: ")
        if username_exists(username):
            print("Username already exists, please login or choose a different username")
            continue 
        save_master_password(username, password)
        print("User Created, please login again to continue")

Username already exists, please login or choose a different username
User found: Sak


## Step 2


In [14]:
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os
def generate_key(password):

    password = password.encode()


    salt = password


    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password)
    return key

def encrypt_data(data, password):
    key = generate_key(password)
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode())
    return encrypted_data

def decrypt_data(encrypted_data, password):
    key = generate_key(password)
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode()
    return decrypted_data

def read_json(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            data = json.load(file)
    else:
        data = []
    return data

def write_json(filename, data):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)



In [16]:
def add_credentials(masterusername, name, username, password):
    filename = f'users/{masterusername}.json'
    data = read_json(filename)
    user = {'name': name, 'username': username, 'password': password}
    data.append(user)
    write_json(filename, data)
    
def delete_credentials(masterusername, username):
    filename = f'users/{masterusername}.json'
    data = read_json(filename)
    data = [user for user in data if user['username'] != username]
    write_json(filename, data)

def update_credentials(masterusername, username, field, new_value):
    filename = f'users/{masterusername}.json'
    data = read_json(filename)
    for user in data:
        if user['username'] == username:
            user[field] = new_value
            break
    write_json(filename, data)
        

In [ ]:
choice1 = input("Enter 1 to add credentials, 2 to delete credentials, 3 to update credentials, 4 to view credentials: ")
if int(choice1) == 1:
    name = input("Enter the name of the credential: ")
    username = input("Enter the username: ")
    password = input("Enter the password: ")
    password = encrypt_data(password, masterpassword)
    add_credentials(username, name, username, password)
    print("Credentials added successfully")
elif int(choice1) == 2:
    username = input("Enter the username of the credential you want to delete: ")
    delete_credentials(masterusername, username)
    print("Credentials deleted successfully")
elif int(choice1) == 3:
    username = input("Enter the username of the credential you want to update: ")
    field = input("Enter the field you want to update (name, username, password): ")
    new_value = input("Enter the new value: ")
    if field == 'password':
        new_value = encrypt_data(new_value, masterpassword)
    update_credentials(masterusername, username, field, new_value)
    print("Credentials updated successfully")